In [2]:
!jupyter nbconvert --to script "D:/Github/TweeterPy/get_tweet_stuff.ipynb" --no-prompt

[NbConvertApp] Converting notebook D:/Github/TweeterPy/get_tweet_stuff.ipynb to script
[NbConvertApp] Writing 7574 bytes to D:\Github\TweeterPy\get_tweet_stuff.py


In [1]:
!pip install tweeterpy

     -------------------------------------- 158.3/158.3 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 123.6/123.6 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 131.5/131.5 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 383.3/383.3 kB 12.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=d2732182128ce4eb307d6aa6b37469c51bc8d1fa2eed9b139e92a1836f035100
  Stored in directory: c:\users\rafit\appdata\local\pip\cache\wheels\e4\62\1d\d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
  Created wheel for demjson3: filename=demjson3-3.0.6-py3-none-any.whl size=75333 sha256=9855a07f6bb315fee13006b81bfa5dde4b6fac62

In [2]:
import json 

In [ ]:
#twitter password  =AJ#~Z+cBUR&7r/ 

In [3]:
from tweeterpy import TweeterPy

In [4]:
twitter = TweeterPy()

2023-09-22 19:33:29,310 [INFO] :: API Updated Successfully.


In [ ]:
# twitter.get_liked_tweets('rngland')

In [ ]:
# twitter.get_friends('rngland', following=True)

In [ ]:
# with open("me.json", "w") as a:
#     json.dump(twitter.get_user_info('rngland'), a)

In [7]:
twitter.logged_in()

2023-09-22 20:14:40,021 [INFO] :: User is authenticated.


True

In [3]:
from liked_tweets import liked_tweets
liked_tweets = liked_tweets.locals()

ImportError: cannot import name 'liked_tweets' from 'liked_tweets' (d:\Github\TweeterPy\liked_tweets.py)

In [91]:
with open('liked_tweets.py', 'r') as file:
    file_contents = file.read()

In [92]:
my_dict = {}
exec(file_contents, my_dict)

In [86]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Markdown

In [93]:
len(my_dict['liked']['data'])

668

In [130]:
rest_ids = []
contains_rest_id = my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']
for i in range(len(my_dict['liked']['data'])):
    if 'rest_id' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        rest_ids.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])
    
rest_ids[0:10]

['1705155679819084179',
 '1700004016506216614',
 '1704989552459579398',
 '1704698971996803138',
 '1704191384435990713',
 '1703960491704377476',
 '1703819665162568007',
 '1703794209327649001',
 '1703259351610511574',
 '1590779807587266561']

In [107]:
from datetime import datetime

In [111]:
created_at = []
for i in range(len(my_dict['liked']['data'])):
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        created_at.append(datetime.strptime(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'], "%a %b %d %H:%M:%S %z %Y").strftime("%Y-%m-%d %H:%M:%S"))
len(created_at)
created_at[0:10]

664

['2023-09-22 09:42:48',
 '2023-09-08 04:31:56',
 '2023-09-21 22:42:41',
 '2023-09-21 03:28:01',
 '2023-09-19 17:51:03',
 '2023-09-19 02:33:33',
 '2023-09-18 17:13:58',
 '2023-09-18 15:32:49',
 '2023-09-17 04:07:29',
 '2022-11-10 18:53:56']

In [180]:
profile_images = []
for i in range(len(my_dict['liked']['data'])):
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        profile_images.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https'])
        
len(profile_images)
profile_images[0:10]


664

['https://pbs.twimg.com/profile_images/1604292546645172224/SL57a00s_normal.png',
 'https://pbs.twimg.com/profile_images/1689852229736697856/aQmWl1ku_normal.jpg',
 'https://pbs.twimg.com/profile_images/1592599455895031808/VM5FWW47_normal.jpg',
 'https://pbs.twimg.com/profile_images/1681138725047136257/5f0aG3LH_normal.jpg',
 'https://pbs.twimg.com/profile_images/1347621377503711233/bHg3ipfD_normal.jpg',
 'https://pbs.twimg.com/profile_images/1677002315813847040/tdq-LQh1_normal.jpg',
 'https://pbs.twimg.com/profile_images/1642647714549710854/QlI3xw3I_normal.jpg',
 'https://pbs.twimg.com/profile_images/1639426426494652416/ZQTBrYsc_normal.jpg',
 'https://pbs.twimg.com/profile_images/1701107606058921984/cQGzyrh3_normal.jpg',
 'https://pbs.twimg.com/profile_images/1563635599906549761/wIum4L8I_normal.jpg']

In [211]:

g_sheet_profile_images = ["=IMAGE(\"" + profile_images[i] + "\"" + ',4 ,45, 45)'  for i in range(len(profile_images))]

g_sheet_profile_images = [image.replace("'", "") for image in g_sheet_profile_images]

g_sheet_profile_images[0:10]





['=IMAGE("https://pbs.twimg.com/profile_images/1604292546645172224/SL57a00s_normal.png",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1689852229736697856/aQmWl1ku_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1592599455895031808/VM5FWW47_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1681138725047136257/5f0aG3LH_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1347621377503711233/bHg3ipfD_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1677002315813847040/tdq-LQh1_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1642647714549710854/QlI3xw3I_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1639426426494652416/ZQTBrYsc_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1701107606058921984/cQGzyrh3_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1563635599906549761/wIum4L8I_normal.jpg",4 ,

In [136]:
screen_names = []
for i in range(len(my_dict['liked']['data'])):
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        screen_names.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name'])
len(screen_names)
screen_names[0:10]


664

['tdinh_me',
 'theJordanNoone',
 'mayfer',
 'cto_junior',
 'gdb',
 'space_tintin',
 'yacineMTB',
 'OODAwiki',
 'shakoistsLog',
 'kepano']

In [177]:
import gspread

credentials = {
    "installed": {
        "client_id": "117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com",
        "project_id": "docs-382418",
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_secret": "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",
        "redirect_uris": ["http://localhost"],
    }
}

In [184]:
gc, authorized_user = gspread.oauth_from_dict(credentials)

sh = gc.open("liked_tweets")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58467%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=05bqG0Ff4OGn1Nxxb2DSu6OpMKnwHb&access_type=offline


In [ ]:
!pip install gspread_dataframe 

In [198]:
from gspread_dataframe import set_with_dataframe

In [199]:
worksheet  = sh.get_worksheet(0)

In [210]:
set_with_dataframe(worksheet, df) 

In [194]:
len((df.columns))

6

In [196]:
sh.add_worksheet(title='h', rows=len(df), cols=len((df.columns))).update(
    [df.columns.values.tolist()] + df.values.tolist()
)

{'spreadsheetId': '1vVIUT1HnLsaAGIRRJbJqI6Z7kVdmaLwUd7Ei-NkPPHA',
 'updatedRange': 'h!A1:F665',
 'updatedRows': 665,
 'updatedColumns': 6,
 'updatedCells': 3990}

In [188]:
worksheet_list = sh.worksheets()
worksheet_names = [i.title for i in worksheet_list]
worksheet_names

['liked_tweets']

In [129]:
tweet_url =['https://twitter.com/' + screen_name[i] + '/status/' + rest_ids[i] for i in range(len(
screen_name))]

In [118]:
full_text = []
for i in range(len(my_dict['liked']['data'])):
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        full_text.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['full_text'])
len(full_text)
full_text[0:10]

664

['My full story: \n\nhttps://t.co/SXBdF7Th2B',
 '@ryxcommar meanwhile the best coders write their powerpoint slides in python with python-pptx and these are not the best coders\n\nhttps://t.co/BUN8kuxvJL',
 'scammers salivating harder by the hour',
 '@up_61347132 @sog_on_bird_app @deepfates https://t.co/E4Z85wBMOq',
 'Get paid to red team our models, using your domain expertise from a variety of fields: https://t.co/iCQT0otZQe https://t.co/Mod3Fx1F83',
 'This book has so much alpha https://t.co/m5p2Y8wiDZ',
 '@AlcorRespecter https://t.co/LLGhsVVPw2',
 'https://t.co/fUFLsQvNDi',
 "I want to share my non-rigorous vibes based approach towards data science modeling of data. A lot of my job as a staff data scientist (i'm staff btw) is doing a vibe-check on technical problems, and helping other data scientists from going down an intractable path.",
 'Load up the brain via podcasts, videos, travel, books, conversations. Defrag the brain in nature or in Obsidian.\n\nInspiration is the feeling 

In [209]:
import pandas as pd

# urls
data = {"rest_ids" : rest_ids, "created_at" : created_at,"g_sheet_profile_images" : g_sheet_profile_images ,"screen_names" :screen_names ,"full_text": full_text,"tweet_url": tweet_url}
df = pd.DataFrame(data)
# df.to_csv('D:/Github/TweeterPy/liked_tweets.csv')

In [183]:
df

,rest_ids,created_at,g_sheet_profile_images,screen_names,full_text,tweet_url
0,1705155679819084179,2023-09-22 09:42:48,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tdinh_me,My full story: \n\nhttps://t.co/SXBdF7Th2B,https://twitter.com/tdinh_me/status/1705155679...
1,1700004016506216614,2023-09-08 04:31:56,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",theJordanNoone,@ryxcommar meanwhile the best coders write the...,https://twitter.com/theJordanNoone/status/1700...
2,1704989552459579398,2023-09-21 22:42:41,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",mayfer,scammers salivating harder by the hour,https://twitter.com/mayfer/status/170498955245...
3,1704698971996803138,2023-09-21 03:28:01,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",cto_junior,@up_61347132 @sog_on_bird_app @deepfates https...,https://twitter.com/cto_junior/status/17046989...
4,1704191384435990713,2023-09-19 17:51:03,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",gdb,"Get paid to red team our models, using your do...",https://twitter.com/gdb/status/170419138443599...
...,...,...,...,...,...,...
659,1587867700231081987,2022-11-02 18:02:15,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",NoahTopper,I’m reading “What Computers Still Can’t Do” an...,https://twitter.com/NoahTopper/status/15878677...
660,1584659361632161792,2022-10-24 21:33:28,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",radshaan,Every single talented engineer (electrical/sof...,https://twitter.com/radshaan/status/1584659361...
661,1552124372231458817,2022-07-27 02:51:02,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",BenHanowell,@PhDemetri @baursafi It’s most popular data ma...,https://twitter.com/BenHanowell/status/1552124...
662,1499838567111143431,2022-03-04 20:05:54,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tyba_latam,"¡Invierte tu dinero, no juegues con él! tyba, ...",https://twitter.com/tyba_latam/status/14998385...


In [146]:
# cards 

# each element in the list is
#  a dictionary. for each dictionary search for title, description, photo_image_full_size_alt_text, thumbnail_image and then get value, string_value and url for photo_image_full_size_alt_text

my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][2]['value']['string_value']


'Today is exactly 2 years since I quit my job and become a full-time indie hacker.'

In [85]:
print(my_dict['liked']['data'][8]['content']['itemContent']['tweet_results']['result']['legacy']['full_text'])

urls = my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']

for i in urls:
    i['expanded_url']

# [i for i in urls['expanded_url']]

screen_name = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name']

rest_id = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['rest_id']

'https://twitter.com/' + screen_name + '/status/' + rest_id

screen_name
rest_id

print(my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['description'])


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['legacy']['created_at']

#  'in_reply_to_screen_name': 'ryxcommar',
# 'in_reply_to_status_id_str': '1700002950012145747',

# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['is_blue_verified']

my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][2]['value']['string_value']



# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values']

# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][4]
# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][0]['value']['string_value']


# card  
    #   {'key': 'title',
    #        'value': {'string_value': 'My solopreneur story: zero to $45K/mo in 2 years',
    #   {'key': 'description',
    #        'value': {'string_value': 'Today is exactly 2 years since I quit my job and become a full-time indie hacker.',
        #   {'key': 'photo_image_full_size_alt_text',
        #    'value': {'string_value': 'Search over large image datasets with natural language and computer vision! - GitHub - deepfates/memery: Search over large image datasets with natural language and computer vision!',
        #      {'key': 'thumbnail_image',
        #    'value': {'image_value': {'height': 200,
        #      'width': 400,
        #      'url': 'https://pbs.twimg.com/card_img/1704513029398745088/aRyYEIOj?format=jpg&name=400x400'},
        #     'type': 'IMAGE'}},
        ##card 

I want to share my non-rigorous vibes based approach towards data science modeling of data. A lot of my job as a staff data scientist (i'm staff btw) is doing a vibe-check on technical problems, and helping other data scientists from going down an intractable path.
Indie hacker.

🧠 @TypingMindApp $30K/mo
📸 @XnapperHQ $6K/mo
🛠 @devutils_app $5.5K/mo
🪄 @blackmagic_so (Sold, was $14K/mo)

💬 https://t.co/9b1SSybSdG


'Today is exactly 2 years since I quit my job and become a full-time indie hacker.'